In [4]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# Suppress Warnings for clean notebook
import warnings
warnings.filterwarnings('ignore')

In [5]:
# read dataset
dataset = pd.read_csv('./Melbourne_housing_FULL.csv')
dataset.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [6]:
dataset.nunique()

Suburb             351
Address          34009
Rooms               12
Type                 3
Price             2871
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom2            15
Bathroom            11
Car                 15
Landsize          1684
BuildingArea       740
YearBuilt          160
CouncilArea         33
Lattitude        13402
Longtitude       14524
Regionname           8
Propertycount      342
dtype: int64

In [7]:
# let's use limited columns which makes more sense for serving our purpose
cols_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount', 
               'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Price']
dataset = dataset[cols_to_use]
dataset.head()

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0,1465000.0


In [8]:
dataset.isna().sum()

Suburb               0
Rooms                0
Type                 0
Method               0
SellerG              0
Regionname           3
Propertycount        3
Distance             1
CouncilArea          3
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
Price             7610
dtype: int64

In [9]:
dataset.head()

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0,1465000.0


In [10]:
# Some feature's missing values can be treated as zero (another class for NA values or absence of that feature)
# like 0 for Propertycount, Bedroom2 will refer to other class of NA values
# like 0 for Car feature will mean that there's no car parking feature with house
cols_to_fill_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']
dataset[cols_to_fill_zero] = dataset[cols_to_fill_zero].fillna(0)

# other continuous features can be imputed with mean for faster results since our focus is on Reducing overfitting
# using Lasso and Ridge Regression
dataset["Landsize"]=dataset['Landsize'].fillna(dataset.Landsize.mean())
dataset['BuildingArea']=dataset['BuildingArea'].fillna(dataset.BuildingArea.mean())

In [11]:
dataset.dropna(inplace=True)

In [12]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price,Suburb_Aberfeldie,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
1,2,4019.0,2.5,2.0,1.0,1.0,202.0,160.2564,1480000.0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,4019.0,2.5,2.0,1.0,0.0,156.0,79.0000,1035000.0,0,...,0,0,0,0,0,0,0,0,1,0
4,3,4019.0,2.5,3.0,2.0,0.0,134.0,150.0000,1465000.0,0,...,0,0,0,0,0,0,0,0,1,0
5,3,4019.0,2.5,3.0,2.0,1.0,94.0,160.2564,850000.0,0,...,0,0,0,0,0,0,0,0,1,0
6,4,4019.0,2.5,3.0,1.0,2.0,120.0,142.0000,1600000.0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
X = dataset.drop('Price', axis=1)
y = dataset['Price']

In [20]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=2)

In [21]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train_X, train_y)

In [22]:
reg.score(test_X,test_y)

0.13853683161537644

In [23]:
train_X

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Suburb_Aberfeldie,Suburb_Airport West,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
12799,2,3593.0,4.4,2.0,1.0,1.0,0.000000,62.0000,0,0,...,0,0,0,0,0,0,0,0,0,0
477,3,4836.0,6.3,0.0,0.0,0.0,593.598993,160.2564,0,0,...,0,0,0,0,1,0,0,0,0,0
29182,3,16166.0,14.7,3.0,2.0,2.0,580.000000,160.2564,0,0,...,0,0,0,0,0,0,0,1,0,0
21339,3,8801.0,8.4,3.0,1.0,2.0,593.598993,160.2564,0,0,...,0,0,0,0,1,0,0,0,0,0
22275,3,11918.0,5.2,3.0,1.0,2.0,212.000000,160.2564,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24177,4,2894.0,10.2,4.0,1.0,0.0,828.000000,160.2564,0,0,...,0,0,0,0,0,0,0,0,0,0
15171,1,4442.0,10.1,1.0,1.0,1.0,0.000000,160.2564,0,0,...,0,0,0,0,0,0,0,0,0,0
8585,3,21650.0,11.2,3.0,1.0,1.0,563.000000,118.0000,0,0,...,0,0,0,0,0,0,0,0,0,0
3351,3,11204.0,7.8,3.0,1.0,1.0,404.000000,160.2564,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
train_y

12799     450000.0
477      2865000.0
29182     580000.0
21339    1250000.0
22275    1280000.0
           ...    
24177    1516000.0
15171     330000.0
8585      787000.0
3351     1030000.0
30337     450000.0
Name: Price, Length: 19070, dtype: float64

In [25]:
test_X

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Suburb_Aberfeldie,Suburb_Airport West,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
32841,4,4497.0,15.2,4.0,1.0,1.0,573.000000,119.0000,0,0,...,0,0,0,0,0,0,0,0,0,0
10813,2,4380.0,5.1,2.0,1.0,1.0,0.000000,160.2564,0,0,...,0,0,0,1,0,0,0,0,0,0
7211,2,6821.0,2.3,2.0,1.0,1.0,215.000000,0.0000,0,0,...,0,0,0,0,0,0,0,0,0,0
17799,1,8870.0,7.0,1.0,1.0,1.0,0.000000,39.0000,0,0,...,0,0,0,0,0,0,0,0,0,0
3397,3,11204.0,7.8,0.0,0.0,0.0,593.598993,160.2564,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20856,4,5336.0,15.5,4.0,2.0,4.0,593.598993,231.0000,0,0,...,0,0,0,0,0,0,0,0,0,0
33317,3,9264.0,7.5,3.0,1.0,1.0,440.000000,160.2564,0,0,...,0,0,0,0,0,0,0,0,0,0
25323,4,21650.0,12.0,4.0,1.0,1.0,736.000000,160.2564,0,0,...,0,0,0,0,0,0,0,0,0,0
4641,2,10412.0,9.2,0.0,0.0,0.0,593.598993,160.2564,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
test_y

32841    1720000.0
10813     555000.0
7211      841000.0
17799     415000.0
3397      800000.0
           ...    
20856     762000.0
33317    1465000.0
25323     850000.0
4641      585000.0
11400     705000.0
Name: Price, Length: 8174, dtype: float64

In [27]:
from sklearn import linear_model
lasso_reg=linear_model.Lasso(alpha=50,max_iter=100,tol=0.1)
lasso_reg.fit(train_X,train_y)

Lasso(alpha=50, max_iter=100, tol=0.1)

In [28]:
lasso_reg.score(test_X,test_y)

0.6636111369404489

In [29]:
lasso_reg.score(train_X,train_y)

0.6766985624766824

In [30]:
from sklearn.linear_model import Ridge
ridge_reg=Ridge(alpha=50,max_iter=100,tol=.1)
ridge_reg.fit(train_X,train_y)

Ridge(alpha=50, max_iter=100, tol=0.1)

In [31]:
ridge_reg.score(test_X,test_y)

0.6670848945194958

In [32]:
ridge_reg.score(train_X,train_y)

0.6622376739684328